## Social desirability experiment data merge

Running this notebook will pull week 3, 4 covid image condition data and week 5 covid image only data together to create an analysis dataset

In [1]:
import numpy as np
import pandas as pd

In [60]:
week3and4 = pd.read_csv('../input/34COVID.csv')
week5 = pd.read_csv('../input/week5.csv')

In [61]:
week3and4.columns==week5.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

In [62]:
week5.columns

Index(['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress',
       'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId',
       'RecipientLastName', 'RecipientFirstName', 'RecipientEmail',
       'ExternalReference', 'LocationLatitude', 'LocationLongitude',
       'DistributionChannel', 'UserLanguage', 'timer_First Click',
       'timer_Last Click', 'timer_Page Submit', 'timer_Click Count', 'Q1',
       'Q2', 'Q3', 'Q3_1', 'Q4', 'Q5', 'Q6', 'Q7_1', 'Q8', 'Control1',
       'Control2', 'Control3', 'Control4', 'A1', 'A2', 'A3', 'A4', 'B1', 'B2',
       'B3', 'B4', 'C1', 'C2', 'C3', 'C4', 'Q9', 'Q9_6_TEXT', 'Q10', 'Q11',
       'Q12', 'Control_1', 'Control_2', 'Control_3', 'Control_4',
       'Treatment_A_1', 'Treatment_A_2', 'Treatment_A_3', 'Treatment_A_4',
       'Treatment_B_1', 'Treatment_B_2', 'Treatment_B_3', 'Treatment_B_4',
       'Treatment_C_1', 'Treatment_C_2', 'Treatment_C_3', 'Treatment_C_4',
       'SC0', 'timeload', 'DeviceIdentifier', 'ResponseID', 'ipa

In [63]:
# manage columns
keep_columns = ['ResponseId', 'Finished', 'Q5', 'Q6',
                'Control1', 'Control2', 'Control3', 'Control4', 
                'A1', 'A2', 'A3', 'A4', 
                'B1', 'B2', 'B3', 'B4', 
                'C1', 'C2', 'C3', 'C4',
                'Q9', 'Q9_6_TEXT', 'Q10', 'Q11', 'Q12']

In [64]:
week3and4 = week3and4[keep_columns]
week5 = week5[keep_columns]
week_all = week3and4.append(week5, ignore_index=True)

In [65]:
week_all.shape

(881, 25)

In [66]:
# drop breakoffs
week_all_completes = week_all.loc[week_all['Finished']==1, :]

In [67]:
week_all_completes['Q9'].value_counts(dropna=False)

1.0    375
2.0    261
6.0     21
NaN      8
5.0      3
Name: Q9, dtype: int64

In [68]:
# keep only those indicated gender as male and female and missing
df = week_all_completes.loc[~np.isin(week_all_completes['Q9'], [5, 6]), :]

#### Examine frequencies

In [69]:
df['Q9'].value_counts(dropna=False)

1.0    375
2.0    261
NaN      8
Name: Q9, dtype: int64

In [70]:
df['Q10'].value_counts(dropna=False)

1.0    467
7.0    164
NaN     11
8.0      2
Name: Q10, dtype: int64

In [71]:
df['Q11'].value_counts(dropna=False)

6.0    214
7.0    189
5.0    106
4.0     63
3.0     42
2.0     17
NaN     11
1.0      2
Name: Q11, dtype: int64

In [72]:
df['Q12'].value_counts(dropna=False)

6.0    248
7.0    163
8.0    125
5.0     89
NaN     12
4.0      7
Name: Q12, dtype: int64

In [73]:
df['Q5'].value_counts(dropna=False)

1.0    304
4.0    174
3.0     78
2.0     46
5.0     38
NaN      4
Name: Q5, dtype: int64

In [74]:
df['Q6'].value_counts(dropna=False)

1.0    281
2.0    263
3.0     96
NaN      4
Name: Q6, dtype: int64

In [75]:
df['Control1'].value_counts(dropna=False)

NaN    481
1.0     99
2.0     64
Name: Control1, dtype: int64

In [76]:
df['Control2'].value_counts(dropna=False)

NaN    481
1.0     83
4.0     80
Name: Control2, dtype: int64

#### Reverse recodes

This step is to reverse the recoded values generated by Qualtrics for all experimental conditions <br>
Yes for any reason (1), Yes but only when neccessary (2), No (3), Unsure (4) <br>
Yes (1), No (3), Unsure (4)

Recodes correction: <br>
Control1: 2=3, 3=4 <br>
Control2, Control3, Control4, A1, A2, A3, A4: 4=3, 5=4 <br>
B1, B2, B3, B4, C2, C4: 4=2, 5=3, 6=4 <br>
C3: 7=2, 8=3, 9=4